In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from textblob import TextBlob as tb
import nltk
import math
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

In [3]:
a_df= pd.read_csv('abusive_words.csv')
s1_df= pd.read_csv('spam0.csv', encoding = 'ISO-8859-1')
s2_df= pd.read_csv('spam1.csv')

In [4]:

s1_df.columns=['label','message','C','D','E']
s2_df.columns=['label','message']

In [5]:
spam1_df=s1_df.drop(['E','D','C'],axis=1)
spam2_df=s2_df
spam2_df.head()

,label,message
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...


In [6]:
spam1_df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
frames=[spam1_df,spam2_df]
mails=pd.concat(frames,ignore_index=True)

In [8]:
mails['label'].value_counts()

ham     9637
spam    1494
Name: label, dtype: int64

In [9]:
mails['label'] = mails['label'].replace('spam', 1)
mails['label'] = mails['label'].replace('ham', 0)


In [10]:
#mails_df.iloc[:,0:2]

In [11]:
totalMails= mails['message'].shape[0];
trainIndex,testIndex=list(),list()
for i in range(mails.shape[0]):
    if np.random.uniform(0,1)< 0.75:
        trainIndex+=[i]
    else:
        testIndex+=[i]
trainData=mails.loc[trainIndex]
testData=mails.loc[testIndex]

In [12]:
trainData.reset_index(inplace=True)
trainData.drop(['index'],axis=1,inplace=True)
trainData.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [116]:
testData.reset_index(inplace=True)
testData.drop(['index'],axis=1,inplace=True)
testData

,label,message
0,0,Even my brother is not like to speak with me. ...
1,0,I'm gonna be home soon and i don't want to tal...
2,1,URGENT! You have won a 1 week FREE membership ...
3,1,"XXXMobileMovieClub: To use your credit, click ..."
4,0,Eh u remember how 2 spell his name... Yes i di...
5,1,England v Macedonia - dont miss the goals/team...
6,0,IÛ÷m going to try for 2 months ha ha only joking
7,0,Just forced myself to eat a slice. I'm really ...
8,0,Lol your always so convincing.
9,0,"Wait that's still not all that clear, were you..."


In [87]:
def process_message(message,lower_case=True,stem=True,stop_words=True,gram=1):
    if(lower_case):
        message=message.lower()   #lowercase
    
    regex = re.compile('[%s]' % re.escape(string.punctuation))   #remove punctuations
    for c in string.punctuation:
        message=message.replace(c,"")
    
    words = word_tokenize(message)   #tokens
    words=[w for w in words if len(w) >2]   #only words with size>2
    w=[]
    if gram>1:     #combination of words
        
        for i in range(len(words)-gram+1):
            w += [''.join(words[i:i+gram])]        
    if stop_words:      #remove stop words
        sw=stopwords.words('english')
        words= [word for word in words if word not in sw]
    if stem:      #perform stemming
        stemmer=PorterStemmer()
        words= [stemmer.stem(word) for word in words]
    words+=w;
    return words

In [88]:
trainData['message'].apply(process_message)

0       [jurong, point, crazi, avail, bugi, great, wor...
1                                   [lar, joke, wif, oni]
2       [free, entri, wkli, comp, win, cup, final, tkt...
3                    [dun, say, earli, hor, alreadi, say]
4       [nah, dont, think, goe, usf, live, around, tho...
5       [freemsg, hey, darl, week, word, back, like, f...
6       [per, request, mell, mell, oru, minnaminungint...
7       [winner, valu, network, custom, select, receiv...
8       [mobil, month, entitl, updat, latest, colour, ...
9       [six, chanc, win, cash, 100, 20000, pound, txt...
10      [ive, search, right, word, thank, breather, pr...
11                                         [date, sunday]
12                                           [kim, watch]
13           [fine, thatåõ, way, feel, thatåõ, way, gota]
14                                 [serious, spell, name]
15                        [pay, first, lar, stock, comin]
16      [aft, finish, lunch, str, lor, ard, smth, lor,...
17            

In [89]:
count_vectorizer=CountVectorizer(analyzer=process_message)
count_vectorizer.fit(trainData['message'])
message4=trainData['message'][3]
message4
# print ("Vocabulary:", count_vectorizer.vocabulary)
#trainData.iloc[:,1]
#print (freq_term_matrix.todense())

'U dun say so early hor... U c already then say...'

In [114]:
message4matrix = count_vectorizer.transform([message4])
print(message4matrix)
#print (count_vectorizer.get_feature_names()[2719])
#print ("Vocabulary:", count_vectorizer.vocabulary_)
#print (freq_term_matrix.todense())

  (0, 994)	1
  (0, 2483)	1
  (0, 2499)	1
  (0, 3459)	1
  (0, 5808)	2
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [91]:
freq_term_matrix=count_vectorizer.transform(trainData['message'])

In [92]:
print ('Shape of Sparse Matrix: ', freq_term_matrix.shape)
print ('Amount of Non-Zero occurences: ', freq_term_matrix.nnz)
print ('sparsity: %.2f%%' % (100.0 * freq_term_matrix.nnz /
                             (freq_term_matrix.shape[0] * freq_term_matrix.shape[1])))

Shape of Sparse Matrix:  (8398, 7740)
Amount of Non-Zero occurences:  66150
sparsity: 0.10%


In [111]:
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)
tfidf4 = tfidf.transform(message4matrix)
print(tfidf.todense())

AttributeError: 'TfidfTransformer' object has no attribute 'todense'

In [113]:
tf_idf_matrix = tfidf.transform(freq_term_matrix)
print (tfidf.idf_[count_vectorizer.vocabulary_['jurong']])
print (tfidf.idf_[count_vectorizer.vocabulary_['fit']])
print (tf_idf_matrix.todense())

8.937255641457519
8.426430017691526
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [95]:
spam_detect_model = MultinomialNB().fit(tf_idf_matrix,trainData['label'])

In [96]:
    print ('Predicted: ',spam_detect_model.predict(tfidf4)[0] )
print ('Expected: ',trainData['label'][3])

Predicted:  0
Expected:  0


In [97]:
all_predictions = spam_detect_model.predict(tf_idf_matrix)
print (all_predictions)

[0 0 1 ... 1 1 0]


In [106]:
trainData.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [99]:
print (classification_report(trainData['label'], all_predictions))

             precision    recall  f1-score   support

          0       0.98      1.00      0.99      7290
          1       1.00      0.89      0.94      1108

avg / total       0.99      0.99      0.98      8398



In [100]:
pipeline = Pipeline([('CountVect',CountVectorizer(analyzer =process_message)),
                    ('tf-idf',TfidfTransformer()),
                    ('classifier',MultinomialNB())])

In [101]:
pipeline.fit(trainData['message'],trainData['label'])

Pipeline(memory=None,
     steps=[('CountVect', CountVectorizer(analyzer=<function process_message at 0x02FBB7C8>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocess...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [102]:
predictions = pipeline.predict(testData['message'])

In [117]:
count_vectorizer.get_feature_names()

['008704050406',
 '0089mi',
 '0121',
 '01223585236',
 '01223585334',
 '0125698789',
 '020603',
 '0207',
 '02070836089',
 '02073162414',
 '02085076972',
 '020903',
 '021',
 '050703',
 '0578',
 '060505',
 '061104',
 '07008009200',
 '07046744435',
 '07090201529',
 '07090298926',
 '07099833605',
 '071104',
 '07123456789',
 '0721072',
 '07732584351',
 '07734396839',
 '07742676969',
 '07753741225',
 '0776xxxxxxx',
 '07786200117',
 '077xxx',
 '07801543489',
 '07808',
 '07808247860',
 '07808726822',
 '07815296484',
 '07821230901',
 '0784987',
 '0789xxxxxxx',
 '0794674629107880867867',
 '07973788240',
 '07xxxxxxxxx',
 '0800',
 '08000407165',
 '08000776320',
 '08000839402',
 '08000930705',
 '08000938767',
 '08001950382',
 '08002888812',
 '08002986030',
 '08002986906',
 '08002988890',
 '08006344447',
 '0808',
 '08081560665',
 '0825',
 '0844',
 '08448350055',
 '08448714184',
 '0845',
 '08450542832',
 '08452810071',
 '08452810073',
 '08452810075over18',
 '0870',
 '08700621170150p',
 '08701213186',


In [76]:
print (classification_report(predictions,testData['label']))

             precision    recall  f1-score   support

          0       1.00      0.98      0.99      2393
          1       0.88      1.00      0.94       340

avg / total       0.99      0.98      0.98      2733



In [82]:
msg='''ar User,

Guys congratulation to all the learners and educators of Eckovation. This year, Prime Minister's Excellence Award for Innovation is given to Unnayan Project of Eckovation. Read the story: Click Here

Celebrating success, launched a free certification course on Introduction to Machine Learning: Click Here

 
'''

In [83]:
print(pipeline.predict([msg]))

[0]


In [84]:
predictions

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)